In [2]:
from thefuzz import fuzz
import json
import spacy
from tqdm import tqdm

In [3]:
vague_match = []

def formatString(input_string): # remove prefix numbers from a label
    stripped_string = input_string.lstrip()
    
    # Check if the stripped string starts with a number
    if stripped_string and stripped_string[0].isdigit():
        # Remove leading numbers
        return stripped_string.lstrip('0123456789 ')
    
    return input_string.lstrip()

def label_fuzzmatch(subs, keyword, threshold=80):
    

    key_value, key_lang = keyword
    key_value = formatString(key_value) # remove the number prefix
    best_match_subject = None
    
    flag = 0

    for sub in subs:
        subject_id = sub['identifier']
        label_dict = sub['labels']
        
        # Flatten labels: include all labels in 'en' and the specified language (if exists)
        # all_labels = labels.get('en', [])  # Default to 'en' labels
        # if key_lang in labels and key_lang != 'en':
        #     all_labels.extend(labels[key_lang])  # Add labels for the specified language
        all_labels = [label for labels in label_dict.values() for label in labels]
        
        sub_flag = 0
        # get the highest matched value of this sub
        for lab in all_labels:
            matched_ratio = fuzz.ratio(key_value.lower(), lab.lower()) # if case sensitive. fuzz.ratio ("water", "water") = 80, short words can have a low match score
            if matched_ratio > sub_flag:
                sub_flag = matched_ratio
        
        # get the highest matched subject and the matched value
        if sub_flag > flag:
            flag = sub_flag
            best_match_subject = subject_id

        
    # check if the matching value with the threshold
    if flag >= threshold:
        #print('find matched label:', key_value, best_match_subject, flag)
        if flag < 100:
            list = [keyword, best_match_subject, flag]
            if list not in vague_match:
                vague_match.append(list)
        return best_match_subject
    else:
        return None


In [4]:
with open('concepts.json', 'r') as file_concept:
    concepts = json.load(file_concept)
print(concepts[0])
with open('keywords.json', 'r') as file_keywords:
    keywords = json.load(file_keywords)
print(keywords[0])


{'identifier': 'https://soilwise-he.github.io/soil-health#AgriculturalAreaUnderSevereErosion', 'relevant_uris': [], 'labels': {'en': ['agricultural area under severe erosion']}}
['040103 agronomy & agriculture', 'en']


## Spacy model

In [ ]:
load_model = spacy.load("en_core_web_sm", disable = ['parser','ner'])


In [ ]:
for i in range(50):
    example = formatString(keywords[i][0])
    print(f"{i} : {example}")

In [12]:
example = formatString(keywords[47][0])

In [ ]:
doc = load_model(example)

In [ ]:
" ".join([token.lemma_ for token in doc])

## Trankit

In [ ]:
from trankit import Pipeline

In [6]:
import trankit
print(trankit.supported_langs)

['afrikaans', 'ancient-greek-perseus', 'ancient-greek', 'arabic', 'armenian', 'basque', 'belarusian', 'bulgarian', 'catalan', 'chinese', 'traditional-chinese', 'classical-chinese', 'croatian', 'czech-cac', 'czech-cltt', 'czech-fictree', 'czech', 'danish', 'dutch', 'dutch-lassysmall', 'english', 'english-gum', 'english-lines', 'english-partut', 'estonian', 'estonian-ewt', 'finnish-ftb', 'finnish', 'french', 'french-partut', 'french-sequoia', 'french-spoken', 'galician', 'galician-treegal', 'german', 'german-hdt', 'greek', 'hebrew', 'hindi', 'hungarian', 'indonesian', 'irish', 'italian', 'italian-partut', 'italian-postwita', 'italian-twittiro', 'italian-vit', 'japanese', 'kazakh', 'korean', 'korean-kaist', 'kurmanji', 'latin', 'latin-perseus', 'latin-proiel', 'latvian', 'lithuanian', 'lithuanian-hse', 'marathi', 'norwegian-nynorsk', 'norwegian-nynorsklia', 'norwegian-bokmaal', 'old-french', 'old-russian', 'persian', 'polish-lfg', 'polish', 'portuguese', 'portuguese-gsd', 'romanian-nonsta

In [9]:
# p = Pipeline('auto')
p = Pipeline('auto', gpu=False, cache_dir='./cache_trankit_models/')
#p = Pipeline('english', gpu=False)

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 2] No such file or directory: 'cache_trankit_models\\xlm-roberta-base\\models--xlm-roberta-base\\.no_exist\\e73636d4f797dec63c3081bb6ed5c7b0bb3f2089\\added_tokens.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 2] No such file or directory: 'cache_trankit_models\\xlm-roberta-base\\models--xlm-roberta-base\\.no_exist\\e73636d4f797dec63c3081bb6ed5c7b0bb3f2089\\special_tokens_map.json'


Loading pretrained XLM-Roberta, this may take a while...


model.safetensors:   2%|1         | 21.0M/1.12G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [9]:
import os

In [ ]:
path = r'./cache_trankit_models/xlm-roberta-base\\models--xlm-roberta-base\\snapshots\\e73636d4f797dec63c3081bb6ed5c7b0bb3f2089\\tokenizer_config.json'

print(f"Path exists: {os.path.exists(path)}")       # Check if the path exists
print(f"Is file: {os.path.isfile(path)}")          # Check if it's a file
print(f"Directory exists: {os.path.isdir(os.path.dirname(path))}")  # Check if the directory exists
dir = os.path.dirname(path)
print(dir)
path2 = '\\\\?\\'+os.path.abspath(path)
print(path2)
print(f"Path exists: {os.path.exists(path2)}")       # Check if the path exists
print(f"Is file: {os.path.isfile(path2)}")          # Check if it's a file
print(f"Directory exists: {os.path.isdir(os.path.dirname(path2))}")  # Check if the directory exists


In [ ]:
path2.startswith("\\\\?\\")

In [ ]:

os.path.exists('\\\\?\\c:\\Users\\vercrume\\OneDrive - Vlaamse overheid - Office 365\\Documenten\\GitHub\\Soilwise Repo\\metadata-augmentation\\keyword-matcher\\cache_trankit_models\\xlm-roberta-base\\models--xlm-roberta-base\\blobs\\6fd4797bc397c3b8b55d6bb5740366b57e6a3ce91c04c77f22aafc0c128e6feb')

In [28]:
keywords[0][0]

'040103 agronomy & agriculture'

In [60]:
keywords

[['040103 agronomy & agriculture', 'en'],
 ['0401 agriculture, forestry, and fisheries', 'en'],
 ['04 agricultural and veterinary sciences', 'en'],
 ['04010301 Horticulture/Soil', 'en'],
 ['Bodem', 'nl'],
 ['DOV', 'nl'],
 ['Databank Ondergrond Vlaanderen', 'nl'],
 ['Departement Omgeving', 'nl'],
 ['Herbruikbaar', 'nl'],
 ['Kosteloos', 'nl'],
 ['Lijst M&R INSPIRE', 'nl'],
 ['Metadata GDI-Vl-conform', 'nl'],
 ['Metadata INSPIRE-conform', 'nl'],
 ['Ondergrond', 'nl'],
 ['Regionaal', 'nl'],
 ['Toegevoegd GDI-Vl', 'nl'],
 ['Vlaamse Open data', 'nl'],
 ['Vlaanderen', 'nl'],
 ['akker', 'nl'],
 ['basic soil properties', 'nl'],
 ['bos', 'nl'],
 ['grasland', 'nl'],
 ['koolstof', 'nl'],
 ['landbouw', 'nl'],
 ['ondergrond', 'nl'],
 ['soil type', 'nl'],
 ['totaal aan organische koolstof', 'nl'],
 ['NIBIS-Metadaten', 'de'],
 ['inspireidentifiziert', 'de'],
 ['0106 biological sciences', 'en'],
 ['Soil', 'en'],
 ['Ultraviolet Rays', 'en'],
 ['01060601 Agronomy/Crops', 'en'],
 ['Water', 'en'],
 ['Seaso

In [66]:
with open('lemma_keywords.json', 'w') as output_file:
	json.dump(lemma_keywords, output_file)

In [68]:
import numpy as np

In [1]:
# Chunk size
n = 20

chunks = []

# Iterate and slice the list
for i in range(0, len(keywords), n):
    chunks.append(keywords[i:i + n])

chunks

# # Convert numpy arrays back to lists
# chunks = [list(chunk) for chunk in chunks]

NameError: name 'keywords' is not defined

In [65]:
lemma_keywords = []
keywords_list = keywords.copy()
for keyword in tqdm(keywords_list):
    text = formatString(keyword[0])
    lemmatized_doc = p.lemmatize(text, is_sent=True)
    sTemp = []
    for token in lemmatized_doc['tokens']:
        if 'expanded' in token.keys():
            for subtoken in token['expanded']:
                sTemp.append(subtoken['lemma']) 
        else:
            sTemp.append(token['lemma'])
    lemma_text = ' '.join(sTemp)
    keyword.extend([lemma_text,lemmatized_doc['lang']])                       
    lemma_keywords.append(keyword)
with open('lemma_keywords.json', 'w') as output_file:
	json.dump(lemma_keywords, output_file)
lemma_keywords

  1%|          | 565/81826 [15:11<36:25:26,  1.61s/it]


KeyboardInterrupt: 

In [56]:
keyword

['Kosteloos', 'nl', 'Kosteloos', 'german', 'Kosteloos', 'german']

In [42]:
lemmatized_doc

{'text': 'Enzimas do solo',
 'tokens': [{'id': 1, 'text': 'Enzimas', 'span': (0, 7), 'lemma': 'enzima'},
  {'id': (2, 3),
   'text': 'do',
   'expanded': [{'id': 2, 'text': 'de', 'lemma': 'de'},
    {'id': 3,
     'text': 'o',
     'feats': 'Definite=Def|Gender=Masc|Number=Sing|PronType=Art',
     'lemma': 'o'}],
   'span': (8, 10)},
  {'id': 4, 'text': 'solo', 'span': (11, 15), 'lemma': 'solo'}],
 'lang': 'portuguese'}

In [44]:
    sTemp = []
    for token in lemmatized_doc['tokens']:
        if 'expanded' in token.keys():
            for subtoken in token['expanded']:
                sTemp.append(subtoken['lemma']) 
        else:
            sTemp.append(token['lemma'])
    lemma_text = ' '.join(sTemp)
    sTemp

['enzima', 'de', 'o', 'solo']

In [41]:
'expanded' in lemmatized_doc['tokens'][1].keys()

True

In [36]:
for token in lemmatized_doc['tokens']:
    print(token)

{'id': 1, 'text': 'Enzimas', 'span': (0, 7), 'lemma': 'enzima'}
{'id': (2, 3), 'text': 'do', 'expanded': [{'id': 2, 'text': 'de', 'lemma': 'de'}, {'id': 3, 'text': 'o', 'feats': 'Definite=Def|Gender=Masc|Number=Sing|PronType=Art', 'lemma': 'o'}], 'span': (8, 10)}
{'id': 4, 'text': 'solo', 'span': (11, 15), 'lemma': 'solo'}


In [33]:
lemmatized_doc

{'text': 'Enzimas do solo',
 'tokens': [{'id': 1, 'text': 'Enzimas', 'span': (0, 7), 'lemma': 'enzima'},
  {'id': (2, 3),
   'text': 'do',
   'expanded': [{'id': 2, 'text': 'de', 'lemma': 'de'},
    {'id': 3,
     'text': 'o',
     'feats': 'Definite=Def|Gender=Masc|Number=Sing|PronType=Art',
     'lemma': 'o'}],
   'span': (8, 10)},
  {'id': 4, 'text': 'solo', 'span': (11, 15), 'lemma': 'solo'}],
 'lang': 'portuguese'}

In [14]:
lemmatized_doc = p.lemmatize(example, is_sent=True)
lemmatized_doc

{'text': 'agriculture, forestry, and fisheries',
 'tokens': [{'id': 1,
   'text': 'agriculture',
   'span': (0, 11),
   'lemma': 'agriculture'},
  {'id': 2, 'text': ',', 'span': (11, 12), 'lemma': ','},
  {'id': 3, 'text': 'forestry', 'span': (13, 21), 'lemma': 'forestry'},
  {'id': 4, 'text': ',', 'span': (21, 22), 'lemma': ','},
  {'id': 5, 'text': 'and', 'span': (23, 26), 'lemma': 'and'},
  {'id': 6, 'text': 'fisheries', 'span': (27, 36), 'lemma': 'fishery'}],
 'lang': 'english'}

In [23]:
' '.join([token['lemma'] for token in lemmatized_doc['tokens']])

'agriculture , forestry , and fishery'